# Stupid word generation with markov chains

**WHY?**
Because I can and most of LLMs are not funny enough

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/short-jokes/shortjokes.csv


In [2]:
!pip install nltk
!pip install markovify

  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18608 sha256=4cffedffc1f85d017f69a604f4912f53dcf89caf42a43a295be89faa437995c1
  Stored in directory: /root/.cache/pip/wheels/ca/8c/c5/41413e24c484f883a100c63ca7b3b0362b7c6f6eb6d7c9cc7f
Successfully built markovify


In [3]:
import pandas as pd
import numpy as np
import nltk
import random
import markovify
from nltk import bigrams, trigrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_csv('/kaggle/input/short-jokes/shortjokes.csv')
tokenized_jokes = [nltk.word_tokenize(jokes.lower()) for jokes in df['Joke']]

In [6]:
text_data = ' '.join(df['Joke'].dropna())
markov_model = markovify.Text(text_data)

In [7]:
def generate_joke_from_prompt(prompt, model, length=100):
    prompt_tokens = word_tokenize(prompt.lower())
    start_word = prompt_tokens[0]
    try:
        joke = model.make_sentence_with_start(start_word, max_overlap_ratio=0.75, tries=100)
    except KeyError:
        joke = None
    if not joke:
        print(f"Prompt '{start_word}' not found. Generating a random joke instead.")
        joke = model.make_sentence(max_overlap_ratio=0.75, tries=100)
    joke = joke.capitalize() if joke else "ARGGGHHH THE VOICES INSIDE MY HEAD IS TOO STRONG SHUT UPPP"
    joke_words = joke.split()
    if len(joke_words) < length:
        while len(joke_words) < length:
            next_word = model.make_sentence(max_overlap_ratio=0.75, tries=100).split()[0]  # Take first word from random sentence
            joke_words.append(next_word)
    elif len(joke_words) > length:
        joke_words = joke_words[:length]
    joke = ' '.join(joke_words)
    
    return f"Did somebody said {prompt}? that reminds me of " + joke

In [8]:
# Example usage
user_prompt = "Why"
generated_joke = generate_joke_from_prompt(user_prompt, markov_model)
print(generated_joke)

Did somebody said Why? that reminds me of Why would you kill a 1 in 3 years. YOU A I I A I I A .. I Why The It's How A Everytime ...would 17038 What A: Guardians In What One Because Who Rent WHAT Have Others A Not I'm I FRAAAAAAPPPBLAPPPPLAAAAAAAPBRENKKKKKKKKKK!!! If What After What Reasons I Soak Because Imagine Because It Does Probably Cause Why I Maybe Sitting What ...now Words A There There Because Which I've Tilda Everyone I'm A In Rough If What Because A: Floyd Glamurai Pretty What My The Me: Their They When I Turns Reddit, A ...because Everyone Because D. Thou


In [9]:
import pickle

model_path = "markov_model.pkl"